# Prepare Datasets

In [1]:
import pandas as pd
from datetime import datetime
import yfinance as yf

## A. Financial Headlines Datasets

Dataset source: [Kaggle Financial Headlines](https://www.kaggle.com/datasets/notlucasp/financial-news-headlines/data)

### A-1: Read & Clean Dataset



In [2]:
# Load the dataset
headline_df = pd.read_csv("../data/cnbc_headlines.csv")

headline_df.head()

,Headlines,Time,Description
0,Jim Cramer: A better way to invest in the Covi...,"7:51 PM ET Fri, 17 July 2020","""Mad Money"" host Jim Cramer recommended buying..."
1,Cramer's lightning round: I would own Teradyne,"7:33 PM ET Fri, 17 July 2020","""Mad Money"" host Jim Cramer rings the lightnin..."
2,NaN,NaN,NaN
3,"Cramer's week ahead: Big week for earnings, ev...","7:25 PM ET Fri, 17 July 2020","""We'll pay more for the earnings of the non-Co..."
4,IQ Capital CEO Keith Bliss says tech and healt...,"4:24 PM ET Fri, 17 July 2020","Keith Bliss, IQ Capital CEO, joins ""Closing Be..."


In [3]:
# Define a function to clean and convert the timestamp
def parse_cnbc_datetime(raw_string):
    try:
        # Remove "ET" and extra whitespace
        raw_string = raw_string.replace("ET", "").strip()
        # Convert to datetime object
        dt = datetime.strptime(raw_string, "%I:%M %p %a, %d %B %Y")
        return dt.date()
    except Exception as e:
        return None

# Apply it
headline_df['Date'] = headline_df['Time'].apply(parse_cnbc_datetime)

# Drop rows with bad or missing dates
headline_df.dropna(subset=['Date'], inplace=True)

headline_df.head()

,Headlines,Time,Description,Date
0,Jim Cramer: A better way to invest in the Covi...,"7:51 PM ET Fri, 17 July 2020","""Mad Money"" host Jim Cramer recommended buying...",2020-07-17
1,Cramer's lightning round: I would own Teradyne,"7:33 PM ET Fri, 17 July 2020","""Mad Money"" host Jim Cramer rings the lightnin...",2020-07-17
3,"Cramer's week ahead: Big week for earnings, ev...","7:25 PM ET Fri, 17 July 2020","""We'll pay more for the earnings of the non-Co...",2020-07-17
4,IQ Capital CEO Keith Bliss says tech and healt...,"4:24 PM ET Fri, 17 July 2020","Keith Bliss, IQ Capital CEO, joins ""Closing Be...",2020-07-17
5,Wall Street delivered the 'kind of pullback I'...,"7:36 PM ET Thu, 16 July 2020","""Look for the stocks of high-quality companies...",2020-07-16


In [4]:
# Keep only useful columns
headline_df = headline_df[['Date', 'Headlines']]  # You can keep 'description' if you want later

# Rename for consistency
headline_df.columns = ['date', 'headline']

# Drop missing or duplicate headlines
headline_df.dropna(subset=['headline'], inplace=True)
headline_df.drop_duplicates(inplace=True)

# Reset index
headline_df.reset_index(drop=True, inplace=True)

headline_df.head()

,date,headline
0,2020-07-17,Jim Cramer: A better way to invest in the Covi...
1,2020-07-17,Cramer's lightning round: I would own Teradyne
2,2020-07-17,"Cramer's week ahead: Big week for earnings, ev..."
3,2020-07-17,IQ Capital CEO Keith Bliss says tech and healt...
4,2020-07-16,Wall Street delivered the 'kind of pullback I'...


### A-2: Basic Statistics of the Dataset

In [5]:
stats_df = headline_df.copy()
print("Date Range:")
print(f"From: {stats_df['date'].min()}  To: {stats_df['date'].max()}")
print(f"Total unique days: {stats_df['date'].nunique()}")

Date Range:
From: 2018-03-01  To: 2020-07-17
Total unique days: 290


In [6]:
headlines_per_day = stats_df.groupby('date').size()
print("\nHeadlines per Day:")
print(f"Average: {headlines_per_day.mean():.2f}")
print(f"Min: {headlines_per_day.min()}  Max: {headlines_per_day.max()}")


Headlines per Day:
Average: 4.40
Min: 1  Max: 11


In [7]:
# Count words and characters
stats_df['headline_word_count'] = stats_df['headline'].apply(lambda x: len(str(x).split()))
stats_df['headline_char_count'] = stats_df['headline'].apply(lambda x: len(str(x)))

print("\nHeadline Length (Words):")
print(stats_df['headline_word_count'].describe().round(1))

print("\nHeadline Length (Characters):")
print(stats_df['headline_char_count'].describe().round(1))


Headline Length (Words):
count    1276.0
mean       12.2
std         2.5
min         2.0
25%        11.0
50%        12.0
75%        14.0
max        21.0
Name: headline_word_count, dtype: float64

Headline Length (Characters):
count    1276.0
mean       74.0
std        13.7
min        14.0
25%        65.0
50%        74.0
75%        84.0
max       105.0
Name: headline_char_count, dtype: float64


### A-3: Save the Cleaned Headline Dataset

In [8]:
headline_df.to_csv("../data/cleaned_cnbc_headlines.csv", index=False)

## B. Market Price Dataset

### B-1: Extract SPY Closing Price from Yahoo!

In [13]:
start_date = headline_df['date'].min()
end_date = headline_df['date'].max()

# Match the date range of your headlines
spy_df = yf.download("SPY", start=start_date, end=end_date, interval="1d")

# Keep only the 'Close' price
spy_df = spy_df[['Close']].reset_index()

# Convert date to match your sentiment dataset format
spy_df['date'] = spy_df['Date'].dt.date
spy_df.drop('Date', axis=1, inplace=True)

# Rename columns for consistency
spy_df.columns = ['spy_close', 'date']

# Check result
print(spy_df.head())

[*********************100%***********************]  1 of 1 completed

    spy_close        date
0  238.321411  2018-03-01
1  239.550018  2018-03-02
2  242.318726  2018-03-05
3  242.932983  2018-03-06
4  242.843948  2018-03-07



/var/folders/tq/2b46y6z13jq7wkv0qgskc1900000gn/T/ipykernel_60672/412361727.py:12: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  spy_df.drop('Date', axis=1, inplace=True)


### B-2: Calculate Market Movement Label

In [14]:
# Calculate daily percentage return
spy_df['spy_return'] = spy_df['spy_close'].pct_change()

# Create target: 1 if market went up next day, 0 if down or flat
spy_df['spy_direction'] = (spy_df['spy_close'].shift(-1) > spy_df['spy_close']).astype(int)

# Drop the last row with NaN values
spy_df.dropna(inplace=True)

# reorder columns
spy_df = spy_df[['date', 'spy_close', 'spy_return', 'spy_direction']]

spy_df.head()

,date,spy_close,spy_return,spy_direction
1,2018-03-02,239.550018,0.005155,1
2,2018-03-05,242.318726,0.011558,1
3,2018-03-06,242.932983,0.002535,0
4,2018-03-07,242.843948,-0.000367,1
5,2018-03-08,244.019043,0.004839,1


### B-3: Save the Dataset

In [15]:
spy_df.to_csv("../data/cleaned_spy_price.csv", index=False)

## C. Extract Other Market Related Features


In [ ]:
# do this part later